In [88]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder 
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [106]:
data = pd.read_csv("hiring.csv")

In [107]:
data.sample(8)

,experience,test_score(out of 10),interview_score(out of 10),salary($)
1,NaN,8.0,6,45000
3,two,10.0,10,65000
5,three,7.0,10,62000
7,eleven,7.0,8,80000
2,five,6.0,7,60000
6,ten,NaN,7,72000
0,NaN,8.0,9,50000
4,seven,9.0,6,70000


In [112]:
data['experience'].fillna(data['experience'].mode()[0],inplace = True)
data['test_score(out of 10)'].fillna(data['test_score(out of 10)'].mean(),inplace = True)

In [113]:
data.sample(8)

,experience,test_score(out of 10),interview_score(out of 10),salary($)
0,eleven,8.000000,9,50000
6,ten,7.857143,7,72000
2,five,6.000000,7,60000
1,eleven,8.000000,6,45000
5,three,7.000000,10,62000
7,eleven,7.000000,8,80000
3,two,10.000000,10,65000
4,seven,9.000000,6,70000


In [114]:
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(data[['experience']])

In [115]:
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['experience']))

In [116]:
data = data.drop('experience',axis=1)
data = pd.concat([data,one_hot_df],axis=1)

In [117]:
data.sample(8)

,test_score(out of 10),interview_score(out of 10),salary($),experience_eleven,experience_five,experience_seven,experience_ten,experience_three,experience_two
0,8.000000,9,50000,1.0,0.0,0.0,0.0,0.0,0.0
4,9.000000,6,70000,0.0,0.0,1.0,0.0,0.0,0.0
5,7.000000,10,62000,0.0,0.0,0.0,0.0,1.0,0.0
3,10.000000,10,65000,0.0,0.0,0.0,0.0,0.0,1.0
6,7.857143,7,72000,0.0,0.0,0.0,1.0,0.0,0.0
7,7.000000,8,80000,1.0,0.0,0.0,0.0,0.0,0.0
2,6.000000,7,60000,0.0,1.0,0.0,0.0,0.0,0.0
1,8.000000,6,45000,1.0,0.0,0.0,0.0,0.0,0.0


In [118]:
x = data.drop('salary($)',axis=1)
scalar = StandardScaler()
x = scalar.fit_transform(x) 

In [120]:
y = data['salary($)']
y

0    50000
1    45000
2    60000
3    65000
4    70000
5    62000
6    72000
7    80000
Name: salary($), dtype: int64

In [121]:
model = LinearRegression()
model.fit(x,y)

LinearRegression()

In [122]:
model.coef_ 

array([-36890.55025901,   2560.04285988,  -5612.59836588, -20370.08030244,
        14906.60384509,   3047.96850443, -10889.47143779,  21520.98212275])

In [123]:
model.intercept_ 

62999.999999999985

In [124]:
theta = np.zeros(x.shape[1] + 1)
learning_rate = 0.0001
iterations = 1000

X_b = np.c_[np.ones((x.shape[0], 1)), x]

In [125]:
def hypothesis(X, theta):
    return X.dot(theta)

In [126]:
def compute_cost(X, y, theta):
    m = len(y)
    cost = (1 / (2 * m)) * np.sum((hypothesis(X, theta) - y) ** 2)
    return cost

In [127]:
def gradient_descent(X, y, theta, learning_rate, iterations):
    m = len(y)
    cost_history = []

    for i in range(iterations):
        gradients = (1 / m) * X.T.dot(hypothesis(X, theta) - y)
        theta = theta - learning_rate * gradients
        
        # Debugging: Print gradients and theta
        print(f"Iteration {i+1}: Gradients = {gradients}, Theta = {theta}")
        
        cost_history.append(compute_cost(X, y, theta))

    return theta, cost_history

In [128]:
# Lower the learning rate
learning_rate = 0.00001  # Significantly reduced

# Optional: Normalize the features
X_normalized = (x - x.mean()) / x.std()
X_b_normalized = np.c_[np.ones((X_normalized.shape[0], 1)), X_normalized]

# Re-initialize theta
theta = np.zeros(X_b_normalized.shape[1])

# Run gradient descent with the normalized data
theta_final, cost_history = gradient_descent(X_b_normalized, y, theta, learning_rate, iterations)
theta_final


Iteration 1: Gradients = [-6.30000000e+04  3.06569670e+01 -4.06894229e+02  3.61478446e+03
  1.13389342e+03 -2.64575131e+03 -3.40168026e+03  3.77964473e+02
 -7.55928946e+02], Theta = [ 6.30000000e-01 -3.06569670e-04  4.06894229e-03 -3.61478446e-02
 -1.13389342e-02  2.64575131e-02  3.40168026e-02 -3.77964473e-03
  7.55928946e-03]
Iteration 2: Gradients = [-6.29993700e+04  3.06847559e+01 -4.06901519e+02  3.61473243e+03
  1.13388279e+03 -2.64572004e+03 -3.40163923e+03  3.77965390e+02
 -7.55915369e+02], Theta = [ 1.25999370e+00 -6.13417229e-04  8.13795748e-03 -7.22951688e-02
 -2.26777621e-02  5.29147135e-02  6.80331949e-02 -7.55929863e-03
  1.51184431e-02]
Iteration 3: Gradients = [-6.29987400e+04  3.07125442e+01 -4.06908808e+02  3.61468040e+03
  1.13387217e+03 -2.64568877e+03 -3.40159821e+03  3.77966307e+02
 -7.55901792e+02], Theta = [ 1.88998110e+00 -9.20542671e-04  1.22070456e-02 -1.08441973e-01
 -3.40164838e-02  7.93716012e-02  1.02049177e-01 -1.13389617e-02
  2.26774611e-02]
Iteration 

array([ 6.26863592e+02, -4.44374735e-01,  4.10516508e+00, -3.58892385e+01,
       -1.12862346e+01,  2.63022093e+01,  3.38127624e+01, -3.78432616e+00,
        7.49205488e+00])